In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #channel_shift_range=10,
    #horizontal_flip=True,
    #vertical_flip=True,
    #fill_mode='nearest',
    #rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/AI_共用/AI_辨識食物類別/tw_food_18',
    target_size=(200,200),
    batch_size=16,
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/AI_共用/AI_辨識食物類別/tw_food_18',
    target_size=(200,200),
    batch_size=16,
    subset='validation')

Found 3113 images belonging to 18 classes.
Found 770 images belonging to 18 classes.


In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

resnet_model = ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))

94781440/94765736 [==============================] - 0s 0us/step


In [5]:
#X_train = preprocess_input(train_generator)

In [6]:
model = Sequential()
model.add(resnet_model)
model.add(Dropout(0.5))
model.add(GlobalAveragePooling2D())
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(18, activation="softmax"))
model.summary()   # 顯示模型摘要資訊

resnet_model.trainable = False

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout (Dropout)           (None, 7, 7, 2048)        0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 50)                102450    
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                918       
                                                        

In [7]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
es = EarlyStopping(monitor = "val_loss", mode = "auto", patience = 3, verbose = 1)
mc = ModelCheckpoint("/content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5", monitor="val_loss", mode="auto", verbose=1, save_best_only=True)

In [9]:
import time
time_start = time.time() #開始計時

In [10]:
history = model.fit_generator(
    train_generator,
    epochs=5000,
    validation_data = validation_generator,
    callbacks = [es, mc])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/5000
195/195 [==============================] - ETA: 0s - loss: 1.7257 - accuracy: 0.5336

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "



Epoch 1: val_loss improved from inf to 0.72510, saving model to /content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5
195/195 [==============================] - 936s 5s/step - loss: 1.7257 - accuracy: 0.5336 - val_loss: 0.7251 - val_accuracy: 0.7909
Epoch 2/5000
195/195 [==============================] - ETA: 0s - loss: 0.8484 - accuracy: 0.7552
Epoch 2: val_loss improved from 0.72510 to 0.50691, saving model to /content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5
195/195 [==============================] - 62s 316ms/step - loss: 0.8484 - accuracy: 0.7552 - val_loss: 0.5069 - val_accuracy: 0.8597
Epoch 3/5000
195/195 [==============================] - ETA: 0s - loss: 0.6453 - accuracy: 0.8028
Epoch 3: val_loss did not improve from 0.50691
195/195 [==============================] - 60s 308ms/step - loss: 0.6453 - accuracy: 0.8028 - val_loss: 0.5427 - val_accuracy: 0.8506
Epoch 4/5000
195/195 [==============================] - ETA: 0s - loss: 0.5124 - accuracy: 0.8365
Epoch 4: val_loss im

In [11]:
time_end = time.time()    #結束計時

time_c= time_end - time_start   #執行所花時間
print('time cost', time_c, 's')

time cost 1671.164784669876 s
